In [21]:
import re

# Exercise 7.1:
# Write a Spark job to count the occurrences of each word in a text file. Document that it works with a small example.

my_text = sc.textFile("../tests/meditations.txt")
split_words = my_text.flatMap(lambda line: re.findall('\w+', line, 0))
map_count = split_words.map(lambda word: (word, 1))
total_count = map_count.reduceByKey(lambda a, b: a + b) 
total_count.collect()

[(u'birth', 1),
 (u'hate', 1),
 (u'arrogant', 1),
 (u'We', 1),
 (u'implicate', 1),
 (u'but', 1),
 (u'Meditations', 1),
 (u'to', 3),
 (u'wrongdoer', 1),
 (u'has', 1),
 (u'Nor', 1),
 (u'And', 1),
 (u'them', 1),
 (u'good', 2),
 (u'they', 1),
 (u'hands', 1),
 (u'not', 1),
 (u'yourself', 1),
 (u'him', 2),
 (u'like', 3),
 (u'dishonest', 1),
 (u'morning', 1),
 (u'these', 1),
 (u't', 1),
 (u'each', 1),
 (u'obstruct', 1),
 (u'upper', 1),
 (u'because', 1),
 (u'From', 1),
 (u'deal', 1),
 (u'people', 1),
 (u'back', 1),
 (u'related', 1),
 (u'relative', 1),
 (u'born', 1),
 (u'are', 2),
 (u'surly', 1),
 (u'divine', 1),
 (u'unnatural', 1),
 (u'obstructions', 1),
 (u'be', 1),
 (u'nature', 1),
 (u'on', 1),
 (u'of', 6),
 (u'recognized', 1),
 (u'turn', 1),
 (u'or', 2),
 (u'own', 1),
 (u'No', 1),
 (u'feel', 2),
 (u'possessing', 1),
 (u'When', 1),
 (u'one', 1),
 (u'feet', 1),
 (u'your', 1),
 (u'rows', 1),
 (u'from', 1),
 (u'angry', 1),
 (u'two', 1),
 (u'But', 1),
 (u'Aurelius', 1),
 (u'tell', 2),
 (u'today'

In [110]:
# Exercise 7.2:
# Write a Spark job that determines if a graph has an Euler tour (all vertices have even degree) where you can assume that the graph you get is connected. 
# This file: 
# https://www.dropbox.com/s/usdi0wpsqm3jb7f/eulerGraphs.txt?dl=0 
# has 5 graphs – for each graph, the first line tells the number of nodes N and the number of edges E. 
# The next E lines tells which two nodes are connected by an edge. 
# Two nodes can be connected by multiple edges.
# It is fine if you split the file into 5 different files. You do not need to keep the node and edge counts in the top of the file.
# Document that it works using a small example.

graphs = ["../tests/euler/1.txt", "../tests/euler/2.txt", "../tests/euler/3.txt",
         "../tests/euler/4.txt", "../tests/euler/5.txt"]

def map_func(x):
    if x[1] % 2 == 0:
        return "All", True
    else:
        return "All", False
    
def reducerEuler(x, y):  # Test if all degress are even
    return x & y
    
for graph_file in graphs:
    graph = sc.textFile(graph_file)
    nodes = graph.flatMap(lambda line: line.split())
    node_degrees = nodes.map(lambda node: (node, 1)) 
    r1 = node_degrees.reduceByKey(lambda x,y: x + y)
    r2 = r1.map(map_func)
    print 'Has Euler tour? ', r2.reduceByKey(reducerEuler).collect()[0][1]



Has Euler tour?  True
Has Euler tour?  False
Has Euler tour?  True
Has Euler tour?  True
Has Euler tour?  False
